In [1]:
import pandas as pd
from datetime import datetime, timedelta
import time
import numpy as np
import qfin as qf

In [2]:
underlying = pd.read_csv(r'E:\NSEI.csv', index_col = 0, parse_dates=['Date'])
nse_xdt = pd.read_csv(r'E:\NSE_xdt.csv', parse_dates=['Date'])
nse_xdt = nse_xdt[nse_xdt['Date'] > nse_xdt['Date'].iloc[2]]

In [3]:
xdt = 'EXPIRY_DT'
ind = 'NIFTY'
stp = 'STRIKE_PR'
smb = 'SYMBOL'
itm = 'INSTRUMENT'
otp = 'OPTION_TYP'
odi = 'OPTIDX'

In [4]:
def file_name_from_date(trade_date):
    return (trade_date.strftime("%Y") + 
            trade_date.strftime("%m") + 
            trade_date.strftime("%d"))

def make_df_from_date(d):
    return pd.read_csv('E:\\NSE\\' + file_name_from_date(d) + '.csv', parse_dates=['EXPIRY_DT'])

def day_to_trade(end_date, distance):
    return underlying.index[underlying.index.get_loc(end_date) - distance]

# def get_stp(t_date, opt_typ, one = True): # has to be calculated on its own based on theta and wingspan
#     return (underlying.loc[t_date]['1sd ' + opt_typ] * one + 
#             underlying.loc[t_date]['1.5sd ' + opt_typ]* (1 - one))

def get_winged_stp(t_date, opt_typ, wings =  1, at_open = True, theta = 1): # has to 
    index_pos = underlying.index.get_loc(t_date)
    if at_open:
        avg_underlying = (underlying.iloc[index_pos - 1]['Open'] + underlying.iloc[index_pos - 1]['Close']) / 2
        avg_vix = (underlying.iloc[index_pos - 1]['vix O'] + underlying.iloc[index_pos - 1]['vix C']) / 2
    else:
        avg_underlying = (underlying.iloc[index_pos]['Open'] + underlying.iloc[index_pos]['Close']) / 2
        avg_vix = (underlying.iloc[index_pos]['vix O'] + underlying.iloc[index_pos]['vix C']) / 2
    percent = avg_vix / ((250 / theta) ** 0.5) / 100
    percent = 1 - percent * ((opt_typ == 'PE') - (opt_typ == 'CE')) * wings
    return mround(avg_underlying * percent)

def no_trades(df, opt_xdt, opt_stp, opt_typ,):
    return (df[(df[smb] == ind) & 
               (df[itm] == odi) & 
               (df[xdt] == opt_xdt) &
               (df[stp] == opt_stp) & 
               (df[otp] == opt_typ)].iloc[0]['HIGH'] == 0)

In [5]:
def mround(x, base=50):
    return int(base * round(float(x)/base))

def b_s_days(theta, depth):
    b = []
    s = []
    for i in range(depth):
        s.append(i) 
        b.append(theta - 1 + i)
    return list(zip(b, s))

In [19]:
def option(df, opt_xdt, opt_stp, opt_typ, at_open = False, enter = True):
    if no_trades(df, opt_xdt, opt_stp, opt_typ):
        index_pos = underlying.index.get_loc(opt_xdt)
        if at_open:
            vix = (underlying.iloc[index_pos - 1]['vix O'] + underlying.iloc[index_pos - 1]['vix C']) / 200
        else:
            vix = (underlying.iloc[index_pos]['vix O'] + underlying.iloc[index_pos]['vix C']) / 200
        underlying_price = underlying.loc[df.iloc[3]['TIMESTAMP']]['Open'] * at_open + underlying.loc[df.iloc[3]['TIMESTAMP']]['Close'] * (1 - at_open)
        days_to_go = underlying.index.get_loc(opt_xdt) - underlying.index.get_loc(df.iloc[3]['TIMESTAMP'])
        risk_free_rate = 0.1
        return (qf.BlackScholesCall(underlying_price, vix, opt_stp, days_to_go/250, risk_free_rate).price * (opt_typ == 'CE') + 
                qf.BlackScholesPut(underlying_price, vix, opt_stp, days_to_go/250, risk_free_rate).price * (opt_typ == 'PE'))
    else:
        return (df[(df[smb] == ind) & 
                       (df[itm] == odi) & 
                       (df[xdt] == opt_xdt) &
                       (df[stp] == opt_stp) & 
                       (df[otp] == opt_typ)].iloc[0]['CLOSE'] * (1 - at_open) + 
                    (df[(df[smb] == ind) & 
                        (df[itm] == odi) & 
                        (df[xdt] == opt_xdt) & 
                        (df[stp] == opt_stp) & 
                        (df[otp] == opt_typ)].iloc[0]['OPEN'] * at_open))

In [44]:
bo, so = True, False
wing = 1.5
theta, depth = 2, 4

start_time = time.time()
trades_across_xdt = []
b_and_s = [(5, 4)]#b_s_days(theta, depth)
for dates in  nse_xdt['Date']: #    [nse_xdt.iloc[2]['Date']]: # 
    pnl = []
    for b_day, s_day in b_and_s:
        b_date = day_to_trade(dates, b_day)
        s_date = day_to_trade(dates, s_day)
        stp_pe = get_winged_stp(b_date, 'PE', wings = wing, at_open = bo)
        stp_ce = get_winged_stp(b_date, 'CE', wings = wing, at_open = bo)
        b_c = -option(make_df_from_date(b_date), dates, stp_ce, 'CE', at_open = bo, enter = True)
        b_p = -option(make_df_from_date(b_date), dates, stp_pe, 'PE', at_open = bo, enter = True)
        s_c = -option(make_df_from_date(s_date), dates, stp_ce, 'CE', at_open = so, enter = False)
        s_p = -option(make_df_from_date(s_date), dates, stp_pe, 'PE', at_open = so, enter = False)
        pnl.append(s_c + s_p - b_c - b_p)
    trades_across_xdt.append(pnl)    
print(time.time() - start_time)

50.00365877151489


In [36]:
pd.DataFrame(trades_across_xdt).to_csv('t2d4boTsoF1_5W.csv')

In [47]:
pd.DataFrame(trades_across_xdt).tail(50)

,0
22,51.75
23,16.20
24,-50.35
25,79.95
26,39.35
27,47.05
28,28.05
29,46.00
30,21.25
31,-28.85


In [10]:
stp_ce

9200

In [39]:
b_s_days(theta, 5)

[(1, 0), (2, 1), (3, 2), (4, 3), (5, 4)]

In [20]:
-option(make_df_from_date(b_date), dates, stp_pe, 'PE', at_open = bo, enter = True)
# make_df_from_date(s_date)

103.26146942095329

In [ ]:
pd.read_csv('E:\\NSE\\' + file_name_from_date(d) + '.csv', parse_dates=['EXPIRY_DT'])

In [30]:
pd.read_csv('E:\\NSE\\' + file_name_from_date(b_date) + '.csv', parse_dates=['EXPIRY_DT'])

AttributeError: 'NotebookFormatter' object has no attribute 'get_result'

      INSTRUMENT     SYMBOL  EXPIRY_DT  STRIKE_PR OPTION_TYP      OPEN  \
0         FUTIDX  BANKNIFTY 2020-05-28        0.0         XX  17923.30   
1         FUTIDX  BANKNIFTY 2020-06-25        0.0         XX  17900.00   
2         FUTIDX  BANKNIFTY 2020-07-30        0.0         XX  17920.85   
3         FUTIDX      NIFTY 2020-05-28        0.0         XX   9068.90   
4         FUTIDX      NIFTY 2020-06-25        0.0         XX   9070.00   
...          ...        ...        ...        ...        ...       ...   
36988     OPTSTK       ZEEL 2020-07-30      190.0         PE      0.00   
36989     OPTSTK       ZEEL 2020-07-30      200.0         PE      0.00   
36990     OPTSTK       ZEEL 2020-07-30      210.0         PE      0.00   
36991     OPTSTK       ZEEL 2020-07-30      220.0         PE      0.00   
36992     OPTSTK       ZEEL 2020-07-30      230.0         PE      0.00   

           HIGH       LOW     CLOSE  SETTLE_PR  CONTRACTS  VAL_INLAKH  \
0      18175.00  17565.60  17644.85   

Timestamp('2020-06-04 00:00:00')

In [481]:
# buy = temp, sell tomorrow evening
# 
# temp = 9
b, s = [3], [2]#range(10), range(10) # [15], [10]#
bo, so = True, False
#day_corrector = not so
ONE = True
# pp = 9850
# cp = 10250
start_time = time.time()
trades_across_xdt = []
#for b in range(1, 10)
for dates in  nse_xdt['Date']: #    [nse_xdt.iloc[2]['Date']]: # 
    trades_for_xdt = []
    for b_day in b:
        b_date = day_to_trade(dates, b_day)
        b_c = -option(make_df_from_date(b_date), dates, get_stp(b_date, 'CE', one = ONE), 'CE', at_open = bo)
        b_p = -option(make_df_from_date(b_date), dates, get_stp(b_date, 'PE', one = ONE), 'PE', at_open = bo)
        pnl = []
#         b_c = -option(make_df_from_date(b_date), dates, cp, 'CE', at_open = bo)
#         b_p = -option(make_df_from_date(b_date), dates, pp, 'PE', at_open = bo)
        for s_day in s:
#        s_day = b_day - 1
            s_date = day_to_trade(dates, s_day)
            s_c = -option(make_df_from_date(s_date), dates, get_stp(b_date, 'CE', one = ONE), 'CE', at_open = so, enter = False)
            s_p = -option(make_df_from_date(s_date), dates, get_stp(b_date, 'PE', one = ONE), 'PE', at_open = so, enter = False)
    #         s_c = -option(make_df_from_date(s_date), dates, cp, 'CE', at_open = so)
    #         s_p = -option(make_df_from_date(s_date), dates, pp, 'PE', at_open = so)
            pnl.append((s_c + s_p - b_c - b_p)/(b_day - s_day + 1))
        trades_for_xdt.append(pnl)
    trades_across_xdt.append(trades_for_xdt)
print(time.time() - start_time)

590.6264553070068


In [493]:
for i in range(70):
    print(trades_across_xdt[i][9][0])
# b_date
# print(trades_across_xdt[1][0][0])

-81.05000000000001
288.4748314033344
68.56574765833705
78.70860111977825
4.299999999999983
-125.11397594016779
71.00000000000003
35.05000000000001
100.70344389795247
26.25
21.299999999999997
23.573639811450334
69.35000000000001
-11.349999999999994
38.238334474425585
143.7
17.121035588110146
20.0
32.900000000000006
-155.00000000000006
33.09999999999998
-15.40000000000002
-51.25
-2.6999999999999886
10.899999999999977
51.05
95.75
17.950000000000003
30.349999999999994
58.5
31.85000000000001
26.30000000000001
-34.500000000000014
55.69999999999999
28.400000000000006
42.69999999999999
41.150000000000034
50.849999999999994
11.25
62.75
-6.350000000000023
-57.599999999999994
39.80000000000004
-43.85000000000002
28.75000000000003
137.55
121.29511773065099
57.14999999999998
34.89999999999998
29.0
29.44999999999999
55.200000000000045
5.0
49.94999999999999
108.40358819276972
31.200000000000003
54.599999999999994
58.8
46.349999999999994
-16.05000000000001
3.249999999999986
47.7
58.650000000000006
13.

In [89]:
underlying.loc[b_date]

Open                 9.544950e+03
High                 9.996050e+03
Low                  9.544350e+03
Close                9.972900e+03
1sd CE               1.020000e+04
1sd PE               9.800000e+03
1.5sd CE             1.030000e+04
1.5sd PE             9.700000e+03
Shares Traded        7.963064e+08
Turnover (Rs. Cr)    3.650577e+04
Name: 2020-06-12 00:00:00, dtype: float64

In [117]:
# def option(df, opt_xdt, opt_stp, opt_typ, at_open = False):
# option(make_df_from_date(b_date), dates, 8500, 'CE')
option(make_df_from_date(b_date), dates, get_stp(b_date, 'PE', one = ONE), 'PE', at_open = bo)

KeyError: 9300.0

In [145]:
# make_df_from_date(b_date)[(make_df_from_date(b_date)['SYMBOL'] == 'NIFTY') & 
#                           (make_df_from_date(b_date)['EXPIRY_DT'] == dates) & 
#                           (make_df_from_date(b_date)['STRIKE_PR'] == 9300)]

option_test(make_df_from_date(b_date), dates, get_stp(b_date, 'PE', one = ONE))

,INSTRUMENT,SYMBOL,EXPIRY_DT,STRIKE_PR,OPTION_TYP,OPEN,HIGH,LOW,CLOSE,SETTLE_PR,CONTRACTS,VAL_INLAKH,OPEN_INT,CHG_IN_OI,TIMESTAMP,Unnamed: 15
2809,OPTIDX,NIFTY,2020-07-02,9700.0,CE,326.40,429.75,294.75,429.75,456.40,19,142.85,225,225,12-JUN-2020,NaN
2897,OPTIDX,NIFTY,2020-07-02,9700.0,PE,265.05,265.05,210.00,210.00,160.95,4,29.83,225,225,12-JUN-2020,NaN


In [144]:
def option_test(df, opt_xdt, opt_stp): #, opt_stp, opt_typ, at_open = False):
    return df[(df[smb] == ind) & 
               (df[itm] == odi) & 
               (df[xdt] == opt_xdt) &
               (df[stp] == opt_stp)]# & 
#                (df[otp] == opt_typ)].iloc[0]['CLOSE'] * (1 - at_open) + 
            
#             df[(df[smb] == ind) & 
#                (df[itm] == odi) & 
#                (df[xdt] == opt_xdt) & 
#                (df[stp] == opt_stp) & 
#                (df[otp] == opt_typ)].iloc[0]['OPEN'] * at_open)

In [279]:
option(make_df_from_date(b_date), dates, get_stp(b_date, 'CE', one = ONE), 'CE', at_open = bo)

0.0

In [267]:
underlying_price = underlying.loc[df.iloc[3]['TIMESTAMP']]['Open']
vix = underlying.loc[df.iloc[3]['TIMESTAMP']]['vix']
days_to_go = underlying.index.get_loc(opt_xdt) - underlying.index.get_loc(df.iloc[3]['TIMESTAMP'])
risk_free_rate = 0.1
(qf.BlackScholesCall(underlying_price, vix, opt_stp, days_to_go/250, risk_free_rate) * (opt_typ == 'CE') + 
        qf.BlackScholesPut(underlying_price, vix, opt_stp, days_to_go/250, risk_free_rate) * (opt_typ == 'PE'))

NameError: name 'opt_xdt' is not defined

In [275]:
qf.BlackScholesCall(underlying_price, vix, 10200, days_to_go/250, risk_free_rate)*('a'=='a')

TypeError: unsupported operand type(s) for *: 'BlackScholesCall' and 'bool'

Timestamp('2020-06-18 00:00:00')